In [ ]:
# Libraries
# ==============================================================================
import os
os.environ["KERAS_BACKEND"] = "torch"  # 'tensorflow', 'jax´ or 'torch'
import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import skforecast
from skforecast.plot import set_dark_theme
from skforecast.datasets import fetch_dataset
from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection import backtesting_forecaster_multiseries

from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('once')

print(f"skforecast version: {skforecast.__version__}")
print(f"keras version: {keras.__version__}")

if keras.__version__ > "3.0":
    print(f"Using backend: {keras.backend.backend()}")
    if keras.backend.backend() == "tensorflow":
        import tensorflow
        print(f"tensorflow version: {tensorflow.__version__}")
    elif keras.backend.backend() == "torch":
        import torch
        print(f"torch version: {torch.__version__}")
    else:
        print("Backend not recognized. Please use 'tensorflow' or 'torch'.")

skforecast version: 0.14.0
keras version: 3.8.0
Using backend: torch
torch version: 2.5.1+cpu


c:\Users\terre\miniconda3\envs\ff_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
df_all = pd.read_csv(r'database\DIM_PLAYER_GAMES.csv')
df_quon = df_all[df_all['PLAYER'].isin(['Saquon Barkley', 'Jalen Hurts'])]

In [39]:
df_quon

,GAME_ORDER,YEAR,DATE,GAME_NUMBER,WEEK,PLAYER,AGE,TEAM,OPPONENT,RESULT,...,PASSING_CMP_PCT,PASSING_YDS,PASSING_TD,PASSING_INT,PASSING_RATE,PASSING_SK,PASSING_YDS1,PASSING_Y_A,PASSING_AY_A,PPR
0,0.0,2018.0,2018-09-09,1.0,1.0,Saquon Barkley,21.212,NYG,JAX,L 15-20,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,20.80
1,1.0,2018.0,2018-09-16,2.0,2.0,Saquon Barkley,21.219,NYG,DAL,L 13-20,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,24.80
2,2.0,2018.0,2018-09-23,3.0,3.0,Saquon Barkley,21.226,NYG,HOU,W 27-22,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,22.70
3,3.0,2018.0,2018-09-30,4.0,4.0,Saquon Barkley,21.233,NYG,NOR,L 18-33,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,22.00
4,4.0,2018.0,2018-10-07,5.0,5.0,Saquon Barkley,21.240,NYG,CAR,L 31-33,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,28.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,72.0,2024.0,2024-11-24,11.0,12.0,Jalen Hurts,26.109,PHI,LAR,W 37-20,...,68.18,179.0,1.0,0.0,108.0,1.0,12.0,8.14,9.05,15.06
2636,73.0,2024.0,2024-12-01,12.0,13.0,Jalen Hurts,26.116,PHI,BAL,W 24-19,...,57.89,118.0,1.0,0.0,93.7,2.0,6.0,6.21,7.26,17.62
2637,74.0,2024.0,2024-12-08,13.0,14.0,Jalen Hurts,26.123,PHI,CAR,W 22-16,...,66.67,108.0,2.0,0.0,110.8,4.0,25.0,5.14,7.05,24.22
2638,75.0,2024.0,2024-12-15,14.0,15.0,Jalen Hurts,26.130,PHI,PIT,W 27-13,...,78.13,290.0,2.0,0.0,125.3,3.0,20.0,9.06,10.31,30.10


In [49]:
df_multi = df_quon.pivot(columns='PLAYER', values='PPR', index='GAME_ORDER').reset_index().head(55)

In [46]:
# Model creation
# ==============================================================================
series = ['Saquon Barkley', 'Jalen Hurts'] 
levels = ['Saquon Barkley', 'Jalen Hurts']   # Features to predict. It can be all the series or less
lags = 10 
steps = 5 
data = df_multi[series].copy()
data_train = df_multi[series].copy()
data_val = df_multi[series].copy()
data_test = df_multi[series].copy()

model = create_and_compile_model(
    series=data_train[['Saquon Barkley', 'Jalen Hurts']],
    levels=levels, 
    lags=lags,
    steps=steps,
    recurrent_layer="LSTM",
    recurrent_units=[100, 50],
    dense_units=[64, 32],
    optimizer=Adam(learning_rate=0.01), 
    loss=MeanSquaredError()
)

forecaster = ForecasterRnn(
    regressor=model,
    levels=levels,
    steps=steps,
    lags=lags,
)

keras version: 3.8.0
Using backend: torch
torch version: 2.5.1+cpu


In [47]:
forecaster.fit(df_multi[['Saquon Barkley', 'Jalen Hurts']])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.2989


In [48]:
predictions = forecaster.predict()
predictions

,Saquon Barkley,Jalen Hurts
50,8.679049,8.575585
51,20.516125,9.383996
52,9.569969,12.642529
53,7.949464,8.511078
54,5.628204,6.326380


In [50]:
df_multi.tail(5)

PLAYER,GAME_ORDER,Jalen Hurts,Saquon Barkley
50,50.0,25.90,17.5
51,51.0,27.26,15.2
52,52.0,29.36,23.0
53,53.0,25.88,5.5
54,54.0,20.90,15.2
